synthetically upsample the data with SMOTE

In [7]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
import numpy as np
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, f1_score, precision_score, recall_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

# 0. Model Training with SMOTE

In [2]:
def get_cross_validation_two_class_normal_distribution(model, X_columns_drop, scale=False):
    """Get cross validated confusion matrix for two class problem"""
    
    # positive scores
    overall_precision = []
    overall_recall = []
    overall_f1 = []
    
    # negative scores
    overall_negative_precision = []
    overall_negative_recall = []
    overall_negative_f1 = []
    
    # loop through all folds
    
    for x in range(5):
        
        # load data
        train = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/one-hot-ecoding/train/cross_validation/training_fraction_normal_distribution/training_'+str(x)+'.csv',sep=";")
        
        # debug
        train_id = train['id']
        del train['Unnamed: 0']
        y_train = train['label']
        del train['label']
        X_train = train.drop(X_columns_drop, axis=1)
        
        # oversample train
        oversample = SMOTE()
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        
        validation = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/one-hot-ecoding/train/cross_validation/validation/val_fold_'+str(x)+'.csv',sep=";")
        # print(validation[validation.id.isin(train_id)])
        del validation['Unnamed: 0']
        y_validation = validation['label']
        del validation['label']
        X_validation = validation.drop(X_columns_drop, axis=1)
        
        # scale if necesary
        if scale:
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = pd.DataFrame(min_max_scaler.fit_transform(X_train))
            X_validation = pd.DataFrame(min_max_scaler.fit_transform(X_validation))
        
        # fit model
        model.fit(X_train, y_train)
        
        # predicted score
        y_predict = model.predict(X_validation)
        f1 = f1_score(y_validation, y_predict)
        precision = precision_score(y_validation, y_predict)
        recall = recall_score(y_validation, y_predict)
        conf_matrix = confusion_matrix(y_validation, y_predict)
        print(conf_matrix)
        print('Positive')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_precision.append(precision)
        overall_recall.append(recall)
        overall_f1.append(f1)
        
        # for negative class (noise), flip labels
        
        precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
        recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
        f1 = (2*precision*recall)/(precision+recall)
        
        print('Negative')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_negative_precision.append(precision)
        overall_negative_recall.append(recall)
        overall_negative_f1.append(f1)
    
    print('---------------------------')
    print('Positive')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_precision), np.std(overall_precision, axis=0),
                                                                           np.mean(overall_recall), np.std(overall_recall, axis=0),
                                                                            np.mean(overall_f1), np.std(overall_f1, axis=0)))
    
    print('Negative')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_negative_precision), np.std(overall_negative_precision, axis=0),
                                                                           np.mean(overall_negative_recall), np.std(overall_negative_recall, axis=0),
                                                                            np.mean(overall_negative_f1), np.std(overall_negative_f1, axis=0)))

# 1.0 Train Logistic Regression

In [4]:
# fit LogReg
log_reg = LogisticRegression(random_state=88)
get_cross_validation_two_class_normal_distribution(log_reg, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[645505 654505]
 [  5392  17454]]
Positive
Precision: 0.025974799057680603 || Recall: 0.7639849426595465 || F1: 0.05024143464713121
Negative
Precision: 0.991716047239425 || Recall: 0.49653848816547563 || F1: 0.6617486123121195


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[669339 630671]
 [  5896  16950]]
Positive
Precision: 0.026172715214608545 || Recall: 0.7419241880416703 || F1: 0.050561772615207015
Negative
Precision: 0.991268225136434 || Recall: 0.5148721932908208 || F1: 0.6777275730352436


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[662948 637062]
 [  5704  17142]]
Positive
Precision: 0.026202835812682283 || Recall: 0.7503282850389565 || F1: 0.050637323683627514
Negative
Precision: 0.9914694041145469 || Recall: 0.5099560772609442 || F1: 0.6735010885565932


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[666827 633183]
 [  5712  17134]]
Positive
Precision: 0.026347150697275327 || Recall: 0.7499781143307362 || F1: 0.05090594699946373
Negative
Precision: 0.9915068122443457 || Recall: 0.5129399004623041 || F1: 0.6761069053290945


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[672871 627141]
 [  5955  16894]]
Positive
Precision: 0.02623149363000458 || Recall: 0.7393759026653245 || F1: 0.05066548305252488
Negative
Precision: 0.9912275015983477 || Recall: 0.5175882991849307 || F1: 0.6800667866697527
---------------------------
Positive
Overall Precision: 0.02618579888245027 (+/- 0.00012091714808921444) || Overall Recall: 0.7491182865472469 (+/- 0.008601045996147878) || Overall F1: 0.050602392199590875 (+/- 0.00021414316993076287)
Negative
Overall Precision: 0.9914375980666199 (+/- 0.00017672797423103382) || Overall Recall: 0.510378991672895 (+/- 0.007354721250310183) || Overall F1: 0.6738301931805607 (+/- 0.006408447431673349)


In [2]:
def get_cross_validation_two_class_normal_distribution_with_undersampling(model, X_columns_drop, scale=False,
                                                                         sampling_strategy_oversample = 0.1,
                                                                         sampling_strategy_undersample = 0.5,
                                                                         k_neighbors=5):
    """Get cross validated confusion matrix for two class problem"""
    
    # positive scores
    overall_precision = []
    overall_recall = []
    overall_f1 = []
    
    # negative scores
    overall_negative_precision = []
    overall_negative_recall = []
    overall_negative_f1 = []
    
    # loop through all folds
    
    for x in range(5):
        
        # load data
        train = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/one-hot-ecoding/train/cross_validation/training_fraction_normal_distribution/training_'+str(x)+'.csv',sep=";")
        
        # debug
        train_id = train['id']
        del train['Unnamed: 0']
        y_train = train['label']
        del train['label']
        X_train = train.drop(X_columns_drop, axis=1)
        
        # oversample train
        oversample = SMOTE(sampling_strategy=sampling_strategy_oversample,
                          k_neighbors=k_neighbors)
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        undersample = RandomUnderSampler(sampling_strategy=sampling_strategy_undersample)
        X_train, y_train = undersample.fit_resample(X_train, y_train)
        
        validation = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/one-hot-ecoding/train/cross_validation/validation/val_fold_'+str(x)+'.csv',sep=";")
        # print(validation[validation.id.isin(train_id)])
        del validation['Unnamed: 0']
        y_validation = validation['label']
        del validation['label']
        X_validation = validation.drop(X_columns_drop, axis=1)
        
        # scale if necesary
        if scale:
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = pd.DataFrame(min_max_scaler.fit_transform(X_train))
            X_validation = pd.DataFrame(min_max_scaler.fit_transform(X_validation))
        
        # fit model
        model.fit(X_train, y_train)
        
        # predicted score
        y_predict = model.predict(X_validation)
        f1 = f1_score(y_validation, y_predict)
        precision = precision_score(y_validation, y_predict)
        recall = recall_score(y_validation, y_predict)
        conf_matrix = confusion_matrix(y_validation, y_predict)
        print(conf_matrix)
        print('Positive')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_precision.append(precision)
        overall_recall.append(recall)
        overall_f1.append(f1)
        
        # for negative class (noise), flip labels
        
        precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
        recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
        f1 = (2*precision*recall)/(precision+recall)
        
        print('Negative')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_negative_precision.append(precision)
        overall_negative_recall.append(recall)
        overall_negative_f1.append(f1)
    
    print('---------------------------')
    print('Positive')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_precision), np.std(overall_precision, axis=0),
                                                                           np.mean(overall_recall), np.std(overall_recall, axis=0),
                                                                            np.mean(overall_f1), np.std(overall_f1, axis=0)))
    
    print('Negative')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_negative_precision), np.std(overall_negative_precision, axis=0),
                                                                           np.mean(overall_negative_recall), np.std(overall_negative_recall, axis=0),
                                                                            np.mean(overall_negative_f1), np.std(overall_negative_f1, axis=0)))

In [5]:
# fit LogReg
log_reg = LogisticRegression(random_state=88)
get_cross_validation_two_class_normal_distribution_with_undersampling(log_reg, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1206090   93920]
 [  17303    5543]]
Positive
Precision: 0.05572926615927531 || Recall: 0.24262452945811083 || F1: 0.09063928247308048
Negative
Precision: 0.9858565481411125 || Recall: 0.9277544018892162 || F1: 0.9559234097764011


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1206955   93055]
 [  17349    5497]]
Positive
Precision: 0.055777660524393216 || Recall: 0.24061104788584434 || F1: 0.0905616237499794
Negative
Precision: 0.9858294998627791 || Recall: 0.928419781386297 || F1: 0.9562637611644194


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1206475   93535]
 [  17319    5527]]
Positive
Precision: 0.055793341543679716 || Recall: 0.24192418804167032 || F1: 0.09067493519703383
Negative
Precision: 0.9858481084234765 || Recall: 0.928050553457281 || F1: 0.9560766208469438


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1207502   92508]
 [  17410    5436]]
Positive
Precision: 0.055501102670913995 || Recall: 0.2379409962356649 || F1: 0.09000745094792616
Negative
Precision: 0.9857867340674269 || Recall: 0.9288405473804048 || F1: 0.9564667740231184


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1206126   93886]
 [  17394    5455]]
Positive
Precision: 0.05491186921814759 || Recall: 0.23874130158869097 || F1: 0.0892871757099599
Negative
Precision: 0.9857836406433895 || Recall: 0.9277806666400003 || F1: 0.9559030755306451
---------------------------
Positive
Overall Precision: 0.05554264802328196 (+/- 0.0003324191218446191) || Overall Recall: 0.24036841264199627 (+/- 0.001794949182191968) || Overall F1: 0.09023409361559596 (+/- 0.0005317547776759785)
Negative
Overall Precision: 0.985820906227637 (+/- 3.0465115533566255e-05) || Overall Recall: 0.9281691901506399 (+/- 0.00041245387196424583) || Overall F1: 0.9561267282683055 (+/- 0.0002136721038752652)


In [6]:
# fit xgboost
xgb = xgb.XGBClassifier(random_state=88)
get_cross_validation_two_class_normal_distribution_with_undersampling(xgb, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

[[1199526  100484]
 [  17274    5572]]
Positive
Precision: 0.052538281662517915 || Recall: 0.24389389827540925 || F1: 0.0864532745806892
Negative
Precision: 0.9858037475345167 || Recall: 0.9227052099599233 || F1: 0.953211406502676
[[1203904   96106]
 [  17464    5382]]
Positive
Precision: 0.05303090020495034 || Recall: 0.23557734395517815 || F1: 0.08657326234175688
Negative
Precision: 0.9857012792213321 || Recall: 0.9260728763624895 || F1: 0.9549571702457941
[[1196336  103674]
 [  17169    5677]]
Positive
Precision: 0.051915391720240324 || Recall: 0.24848988882080014 || F1: 0.08588697171645347
Negative
Precision: 0.9858517270221383 || Recall: 0.9202513826816717 || F1: 0.9519227058521632
[[1202126   97884]
 [  17423    5423]]
Positive
Precision: 0.0524940226702934 || Recall: 0.23737196883480696 || F1: 0.08597496690526583
Negative
Precision: 0.9857135711644223 || Recall: 0.9247051945754263 || F1: 0.9542352451361529
[[1197529  102483]
 [  17322    5527]]
Positive
Precision: 0.051171187852

In [8]:
# fit mlpclassifier
mlp = MLPClassifier(random_state=88)
get_cross_validation_two_class_normal_distribution_with_undersampling(mlp, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

[[1188653  111357]
 [  17124    5722]]
Positive
Precision: 0.048872983199378196 || Recall: 0.2504595990545391 || F1: 0.08178667143112381
Negative
Precision: 0.9857983690184835 || Recall: 0.9143414281428605 || F1: 0.9487262883876403


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[[1192685  107325]
 [  17476    5370]]
Positive
Precision: 0.047650738719552777 || Recall: 0.23505208789284776 || F1: 0.0792380165411204
Negative
Precision: 0.9855589462889649 || Recall: 0.9174429427465942 || F1: 0.9502818732269634
[[1203809   96201]
 [  17773    5073]]
Positive
Precision: 0.05009183008472066 || Recall: 0.2220520003501707 || F1: 0.08174347405736383
Negative
Precision: 0.9854508334274735 || Recall: 0.9259998000015385 || F1: 0.9548007766522102
[[1191579  108431]
 [  17417    5429]]
Positive
Precision: 0.047681363077463554 || Recall: 0.23763459686597216 || F1: 0.0794259213202054
Negative
Precision: 0.9855938315759523 || Recall: 0.9165921800601534 || F1: 0.9498414910127755
[[1198206  101806]
 [  17537    5312]]
Positive
Precision: 0.04959017158647473 || Recall: 0.2324828220053394 || F1: 0.08174382727923242
Negative
Precision: 0.9855750763113585 || Recall: 0.9216884151838598 || F1: 0.9525617558148549
---------------------------
Positive
Overall Precision: 0.0487774173335179

In [11]:
# fit naive bayes
nb = GaussianNB()
get_cross_validation_two_class_normal_distribution_with_undersampling(nb, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

[[ 276849 1023161]
 [   2227   20619]]
Positive
Precision: 0.019754162754603462 || Recall: 0.9025212290991859 || F1: 0.03866209899252409
Negative
Precision: 0.9920200948845476 || Recall: 0.21295913108360706 || F1: 0.35064461340294323
[[ 282412 1017598]
 [   2234   20612]]
Positive
Precision: 0.019853401527629284 || Recall: 0.9022148297294931 || F1: 0.03885186078774353
Negative
Precision: 0.9921516550381878 || Recall: 0.21723832893593126 || F1: 0.3564331943336598
[[ 209068 1090942]
 [   1495   21351]]
Positive
Precision: 0.019195481766045457 || Recall: 0.9345618489013394 || F1: 0.03761830049007214
Negative
Precision: 0.9928999871772344 || Recall: 0.16082030138229705 || F1: 0.2768062185673913
[[ 200247 1099763]
 [   1450   21396]]
Positive
Precision: 0.019083823079509687 || Recall: 0.9365315591350784 || F1: 0.03740543092031941
Negative
Precision: 0.9928109986762321 || Recall: 0.1540349689617772 || F1: 0.266692503930527
[[ 268463 1031549]
 [   2061   20788]]
Positive
Precision: 0.01975412

In [16]:
# fit decision tree
dt = DecisionTreeClassifier()
get_cross_validation_two_class_normal_distribution_with_undersampling(dt, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

[[1188433  111577]
 [  17526    5320]]
Positive
Precision: 0.04551014996107684 || Recall: 0.23286352096647114 || F1: 0.07613977086508804
Negative
Precision: 0.9854671676234432 || Recall: 0.9141721986753948 || F1: 0.9484818048427573
[[1191397  108613]
 [  17758    5088]]
Positive
Precision: 0.0447489467990607 || Recall: 0.2227085704280837 || F1: 0.07452379034325178
Negative
Precision: 0.9853137108145771 || Recall: 0.9164521811370682 || F1: 0.9496362335677407
[[1188265  111745]
 [  17599    5247]]
Positive
Precision: 0.0448492204595186 || Recall: 0.2296682132539613 || F1: 0.07504397946194882
Negative
Precision: 0.9854054851956772 || Recall: 0.9140429689002393 || F1: 0.9483836777108506
[[1188947  111063]
 [  17611    5235]]
Positive
Precision: 0.04501367177423515 || Recall: 0.2291429571916309 || F1: 0.07524578853561778
Negative
Precision: 0.985403934166447 || Recall: 0.9145675802493827 || F1: 0.948665266611558
[[1190985  109027]
 [  17728    5121]]
Positive
Precision: 0.04486280968567123 

In [1]:
undersampling_parameters = [0.1, 0.3, 0.5]
oversampling_parameters = [0.1, 0.3, 0.5]

In [2]:
sampling_parameters = []
for undersampling in undersampling_parameters:
    for oversampling in oversampling_parameters:
        sampling_parameters.append([undersampling, oversampling])
sampling_parameters

[[0.1, 0.1],
 [0.1, 0.3],
 [0.1, 0.5],
 [0.3, 0.1],
 [0.3, 0.3],
 [0.3, 0.5],
 [0.5, 0.1],
 [0.5, 0.3],
 [0.5, 0.5]]

In [6]:
# loop through all sampling parameters
for parameter in sampling_parameters:
    print(f'start with parameters {str(parameter)}')
    
    log_reg = LogisticRegression(random_state=88)
    get_cross_validation_two_class_normal_distribution_with_undersampling(log_reg, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'],
                                                                         parameter[0], parameter[1])
    
    print('------------------------------------------')
    print('------------------------------------------')

start with parameters [0.1, 0.1]


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1213093   86917]
 [  17528    5318]]
Positive
Precision: 0.05765707161055998 || Recall: 0.2327759782894161 || F1: 0.09242185938599769
Negative
Precision: 0.9857567845827432 || Recall: 0.9331412835285883 || F1: 0.9587276849133675


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1207381   92629]
 [  17368    5478]]
Positive
Precision: 0.0558369943021395 || Recall: 0.23977939245382124 || F1: 0.09058063876051027
Negative
Precision: 0.9858191351860667 || Recall: 0.9287474711732987 || F1: 0.956432673375954


...

[[1200214   99796]
 [  17130    5716]]
Positive
Precision: 0.054173932822806886 || Recall: 0.2501969710233739 || F1: 0.08906340080088501
Negative
Precision: 0.9859283817885495 || Recall: 0.9232344366581795 || F1: 0.9535520232752326


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1207172   92838]
 [  17411    5435]]
Positive
Precision: 0.05530511941224955 || Recall: 0.23789722489713735 || F1: 0.08974644770845201
Negative
Precision: 0.9857820988859065 || Recall: 0.9285867031792063 || F1: 0.9563299906163093


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1207337   92675]
 [  17477    5372]]
Positive
Precision: 0.0547900496700562 || Recall: 0.23510875749485755 || F1: 0.0888697723663314
Negative
Precision: 0.9857308946501264 || Recall: 0.9287121965028016 || F1: 0.9563724391304589
---------------------------
Positive
Overall Precision: 0.054848777647086885 (+/- 0.0005072917526170839) || Overall Recall: 0.24295977128361654 (+/- 0.006261211833642298) || Overall F1: 0.08947969378585734 (+/- 0.0005015098729883225)
Negative
Overall Precision: 0.9858422389159974 (+/- 8.030531694164473e-05) || Overall Recall: 0.9264035111384693 (+/- 0.0024514760037320235) || Overall F1: 0.9551973313573379 (+/- 0.0012680776338425599)
------------------------------------------
------------------------------------------


In [4]:
# loop through different neighbors
for k_neighbor in range(2,10):
    print(f'start with k_neighbor = {str(k_neighbor)}')
    
    log_reg = LogisticRegression(random_state=88)
    get_cross_validation_two_class_normal_distribution_with_undersampling(log_reg, 
                                                                          ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'],
                                                                         k_neighbors=k_neighbor)
    
    print('------------------------------------------')
    print('------------------------------------------')

start with k_neighbor = 2


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1214868   85142]
 [  17570    5276]]
Positive
Precision: 0.0583512132539981 || Recall: 0.23093758207125975 || F1: 0.09316287611244525
Negative
Precision: 0.9857437047543163 || Recall: 0.9345066576410951 || F1: 0.9594416153855875


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1205511   94499]
 [  17284    5562]]
Positive
Precision: 0.055586092483585016 || Recall: 0.24345618489013393 || F1: 0.09050745685762406
Negative
Precision: 0.9858651695500881 || Recall: 0.9273090206998408 || F1: 0.955690986818244


...

[[1200529   99481]
 [  17114    5732]]
Positive
Precision: 0.05447995970079743 || Recall: 0.2508973124398144 || F1: 0.0895212363051406
Negative
Precision: 0.9859449773045137 || Recall: 0.9234767424865963 || F1: 0.9536890111544364


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1207795   92215]
 [  17425    5421]]
Positive
Precision: 0.05552255315662256 || Recall: 0.23728442615775192 || F1: 0.08998854600687238
Negative
Precision: 0.9857780643476274 || Recall: 0.929065930262075 || F1: 0.9565821727129806


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1200103   99909]
 [  17197    5652]]
Positive
Precision: 0.053542501492028306 || Recall: 0.24736312311260886 || F1: 0.08803052721750643
Negative
Precision: 0.9858728333196418 || Recall: 0.9231476324833925 || F1: 0.9534797434724023
---------------------------
Positive
Overall Precision: 0.05484561422170327 (+/- 0.0007876096536318192) || Overall Recall: 0.2451830334468236 (+/- 0.006127700933415212) || Overall F1: 0.08962206201410353 (+/- 0.0008449785404629255)
Negative
Overall Precision: 0.985872805898483 (+/- 7.553508589033463e-05) || Overall Recall: 0.9257104412588728 (+/- 0.002715856958788469) || Overall F1: 0.9548427465923857 (+/- 0.0014132407637239006)
------------------------------------------
------------------------------------------


In [5]:
def get_cross_validation_two_class_normal_distribution_BORDERLINE(model, X_columns_drop, scale=False):
    """Get cross validated confusion matrix for two class problem"""
    
    # positive scores
    overall_precision = []
    overall_recall = []
    overall_f1 = []
    
    # negative scores
    overall_negative_precision = []
    overall_negative_recall = []
    overall_negative_f1 = []
    
    # loop through all folds
    
    for x in range(5):
        
        # load data
        train = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/one-hot-ecoding/train/cross_validation/training_fraction_normal_distribution/training_'+str(x)+'.csv',sep=";")
        
        # debug
        train_id = train['id']
        del train['Unnamed: 0']
        y_train = train['label']
        del train['label']
        X_train = train.drop(X_columns_drop, axis=1)
        
        # oversample train
        oversample = BorderlineSMOTE()
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        
        validation = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/one-hot-ecoding/train/cross_validation/validation/val_fold_'+str(x)+'.csv',sep=";")
        # print(validation[validation.id.isin(train_id)])
        del validation['Unnamed: 0']
        y_validation = validation['label']
        del validation['label']
        X_validation = validation.drop(X_columns_drop, axis=1)
        
        # scale if necesary
        if scale:
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = pd.DataFrame(min_max_scaler.fit_transform(X_train))
            X_validation = pd.DataFrame(min_max_scaler.fit_transform(X_validation))
        
        # fit model
        model.fit(X_train, y_train)
        
        # predicted score
        y_predict = model.predict(X_validation)
        f1 = f1_score(y_validation, y_predict)
        precision = precision_score(y_validation, y_predict)
        recall = recall_score(y_validation, y_predict)
        conf_matrix = confusion_matrix(y_validation, y_predict)
        print(conf_matrix)
        print('Positive')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_precision.append(precision)
        overall_recall.append(recall)
        overall_f1.append(f1)
        
        # for negative class (noise), flip labels
        
        precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
        recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
        f1 = (2*precision*recall)/(precision+recall)
        
        print('Negative')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_negative_precision.append(precision)
        overall_negative_recall.append(recall)
        overall_negative_f1.append(f1)
    
    print('---------------------------')
    print('Positive')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_precision), np.std(overall_precision, axis=0),
                                                                           np.mean(overall_recall), np.std(overall_recall, axis=0),
                                                                            np.mean(overall_f1), np.std(overall_f1, axis=0)))
    
    print('Negative')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_negative_precision), np.std(overall_negative_precision, axis=0),
                                                                           np.mean(overall_negative_recall), np.std(overall_negative_recall, axis=0),
                                                                            np.mean(overall_negative_f1), np.std(overall_negative_f1, axis=0)))

In [6]:
# test borderline smote
log_reg = LogisticRegression(random_state=88)
get_cross_validation_two_class_normal_distribution_BORDERLINE(log_reg, 
                                                            ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1041151  258859]
 [  13368    9478]]
Positive
Precision: 0.03532125647972512 || Recall: 0.41486474656394995 || F1: 0.06509995432425657
Negative
Precision: 0.987323130261285 || Recall: 0.8008792240059692 || F1: 0.8843815472223957


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1126512  173498]
 [  15904    6942]]
Positive
Precision: 0.03847262247838617 || Recall: 0.30386063205812835 || F1: 0.06829786606062395
Negative
Precision: 0.9860786263497711 || Recall: 0.8665410266074877 || F1: 0.922453331237057


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1001607  298403]
 [  12683   10163]]
Positive
Precision: 0.03293622758178153 || Recall: 0.44484811345530945 || F1: 0.061331514851604645
Negative
Precision: 0.9874956866379437 || Recall: 0.7704609964538734 || F1: 0.8655809532039925


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1020358  279652]
 [  13187    9659]]
Positive
Precision: 0.0333862175997456 || Recall: 0.42278735883743324 || F1: 0.06188552555284681
Negative
Precision: 0.9872410006337412 || Recall: 0.7848847316559103 || F1: 0.8745094930267339


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1131802  168210]
 [  15286    7563]]
Positive
Precision: 0.04302708607123961 || Recall: 0.33099916845376165 || F1: 0.07615470592381508
Negative
Precision: 0.9866740825464132 || Recall: 0.8706088866871998 || F1: 0.9250149156144007
---------------------------
Positive
Overall Precision: 0.036628682042175606 (+/- 0.003746914668238146) || Overall Recall: 0.3834720038737165 (+/- 0.05547900005638069) || Overall F1: 0.06655391334262942 (+/- 0.0054110604626359)
Negative
Overall Precision: 0.9869625052858308 (+/- 0.0005208304976080688) || Overall Recall: 0.8186749730820881 (+/- 0.041883999583528835) || Overall F1: 0.894388048060916 (+/- 0.024701420442030372)


In [10]:
def get_cross_validation_two_class_normal_distribution_with_undersampling(model, X_columns_drop, scale=False,
                                                                         sampling_strategy_oversample = 0.1,
                                                                         sampling_strategy_undersample = 0.5,
                                                                         k_neighbors=5):
    """Get cross validated confusion matrix for two class problem"""
    
    # positive scores
    overall_precision = []
    overall_recall = []
    overall_f1 = []
    
    # negative scores
    overall_negative_precision = []
    overall_negative_recall = []
    overall_negative_f1 = []
    
    # loop through all folds
    
    for x in range(5):
        
        # load data
        train = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/cross_validation/training_fraction_normal_distribution/training_'+str(x)+'.csv',sep=";", index_col=0)      
        # debug
        train_id = train['id']
        y_train = train['label']
        del train['label']
        X_train = train.drop(X_columns_drop, axis=1)
        
        # oversample train
        oversample = SMOTE(sampling_strategy=sampling_strategy_oversample,
                          k_neighbors=k_neighbors)
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        undersample = RandomUnderSampler(sampling_strategy=sampling_strategy_undersample)
        X_train, y_train = undersample.fit_resample(X_train, y_train)
        
        validation = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/cross_validation/validation/validation_'+str(x)+'.csv',sep=";", index_col=0)
        # print(validation[validation.id.isin(train_id)])
        y_validation = validation['label']
        del validation['label']
        X_validation = validation.drop(X_columns_drop, axis=1)
        
        # scale if necesary
        if scale:
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = pd.DataFrame(min_max_scaler.fit_transform(X_train))
            X_validation = pd.DataFrame(min_max_scaler.fit_transform(X_validation))
        
        # fit model
        model.fit(X_train, y_train)
        
        # predicted score
        y_predict = model.predict(X_validation)
        f1 = f1_score(y_validation, y_predict)
        precision = precision_score(y_validation, y_predict)
        recall = recall_score(y_validation, y_predict)
        conf_matrix = confusion_matrix(y_validation, y_predict)
        print(conf_matrix)
        print('Positive')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_precision.append(precision)
        overall_recall.append(recall)
        overall_f1.append(f1)
        
        # for negative class (noise), flip labels
        
        precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
        recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
        f1 = (2*precision*recall)/(precision+recall)
        
        print('Negative')
        print('Precision: {} || Recall: {} || F1: {}'.format(precision, recall, f1))
        
        overall_negative_precision.append(precision)
        overall_negative_recall.append(recall)
        overall_negative_f1.append(f1)
    
    print('---------------------------')
    print('Positive')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_precision), np.std(overall_precision, axis=0),
                                                                           np.mean(overall_recall), np.std(overall_recall, axis=0),
                                                                            np.mean(overall_f1), np.std(overall_f1, axis=0)))
    
    print('Negative')
    print('Overall Precision: {} (+/- {}) || Overall Recall: {} (+/- {}) || Overall F1: {} (+/- {})'.format(
                                                                            np.mean(overall_negative_precision), np.std(overall_negative_precision, axis=0),
                                                                           np.mean(overall_negative_recall), np.std(overall_negative_recall, axis=0),
                                                                            np.mean(overall_negative_f1), np.std(overall_negative_f1, axis=0)))

In [11]:
# fit LogReg
log_reg = LogisticRegression(random_state=88)
get_cross_validation_two_class_normal_distribution_with_undersampling(log_reg, ['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id', 'pids'])

/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1287061   12949]
 [  21465    1381]]
Positive
Precision: 0.09637124912770412 || Recall: 0.06044821850652193 || F1: 0.07429524424359801
Negative
Precision: 0.9835960462382864 || Recall: 0.9900393073899432 || F1: 0.9868071592648137


/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1288445   11565]
 [  21606    1240]]
Positive
Precision: 0.09683717297930496 || Recall: 0.05427645977413989 || F1: 0.06956326610754256
Negative
Precision: 0.9835075123029562 || Recall: 0.9911039145852725 || F1: 0.9872911016255942


/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1286491   13519]
 [  21547    1299]]
Positive
Precision: 0.08766365231475233 || Recall: 0.05685896874726429 || F1: 0.06897833474936278
Negative
Precision: 0.9835272369762958 || Recall: 0.9896008492242367 || F1: 0.9865546953123563


/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1288352   11658]
 [  21714    1132]]
Positive
Precision: 0.08850664581704457 || Recall: 0.04954915521316642 || F1: 0.06353126052306657
Negative
Precision: 0.9834252625440245 || Recall: 0.9910323766740256 || F1: 0.9872141654112754


/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1288845   11167]
 [  21786    1063]]
Positive
Precision: 0.08691741618969746 || Recall: 0.04652282375596306 || F1: 0.06060606060606061
Negative
Precision: 0.9833774723778088 || Recall: 0.9914100792915758 || F1: 0.9873774391979294
---------------------------
Positive
Overall Precision: 0.09125922728570067 (+/- 0.004395507551593891) || Overall Recall: 0.053531125199411125 (+/- 0.004988960216721335) || Overall F1: 0.0673948332459261 (+/- 0.004813312334850635)
Negative
Overall Precision: 0.9834867060878743 (+/- 7.712098610613848e-05) || Overall Recall: 0.9906373054330107 (+/- 0.0006932283830241917) || Overall F1: 0.9870489121623937 (+/- 0.0003151473127851246)
